In [1]:
import pandas as pd
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from sklearn.cross_validation import train_test_split
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

C:\Users\Nurvirta\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#df = pd.read_csv('New_Classes.csv', index_col=0)
df = pd.read_csv('merged_data_1.1.csv', index_col=0)
df.head()

,id,BoroCT2010,n_report,DBN,school_distance,n_school,subway_distance,n_tobacco,Enrollment,Student Achievement Rating,...,Percent Hispanic,Percent Asian,% Female,% Poverty,MHI,Total Population,type,latitude,longitude,have_sold
0,19157,5000900,0.0,24Q600,2826.764130,0.0,79.859155,3,1575,Approaching Target,...,0.757,0.112,0.377778,0.811429,56192,2112,GROCERY SUPERMARKET,40.640561,-74.082207,0
1,18026,5000900,0.0,10X225,1703.127328,1.0,924.970954,3,427,Approaching Target,...,0.700,0.009,0.674603,1.000000,56192,2112,CONVENIENCE STORE,40.641998,-74.078975,0
2,12113,5000900,0.0,28Q690,2929.402252,0.0,5998.612261,3,589,Approaching Target,...,0.195,0.104,0.463497,0.728353,56192,2112,GROCERY SUPERMARKET,40.640561,-74.082207,0
3,13601,5007400,0.0,15K448,1740.529846,0.0,2489.816362,2,415,Exceeding Target,...,0.422,0.046,0.508100,0.684831,65559,4693,GROCERY SUPERMARKET,40.591647,-74.067662,0
4,10482,5007400,0.0,23K493,630.587028,0.0,941.382096,2,307,Meeting Target,...,0.114,0.010,0.387622,0.856678,65559,4693,GROCERY SUPERMARKET,40.595713,-74.063245,0


In [3]:
df.columns

Index([u'id', u'BoroCT2010', u'n_report', u'DBN', u'school_distance',
       u'n_school', u'subway_distance', u'n_tobacco', u'Enrollment',
       u'Student Achievement Rating', u'Percent White', u'Percent Black',
       u'Percent Hispanic', u'Percent Asian', u'% Female', u'% Poverty',
       u'MHI', u'Total Population', u'type', u'latitude', u'longitude',
       u'have_sold'],
      dtype='object')

In [39]:
# df['class'] = df['n_report'].apply(lambda x: 0 if x == 0 
#                                     else 1 if x == 1 
#                                     else 2 if x == 2 
#                                     else 3 if x >2 and x<=6 else 4)

attributes = ['have_sold', 'MHI','Total Population', 'n_tobacco', 'school_distance', 'subway_distance', 'Enrollment', 
              '% Female', 'n_school', '% Poverty',
             'Percent White', 'Percent Black', 'Percent Hispanic', 'Percent Asian'
             ]
# attributes = ['have_sold', 'MHI','Total Population', 'n_tobacco', 'school_distance', 'subway_distance', 
#               'n_school', 'Enrollment', 'Percent White', 'Percent White', 'Percent Black', 'Percent Hispanic',
#               'Percent Asian', '% Female', '% Poverty' ]

X = df[attributes].copy()

# comment below
X.rename(columns={'have_sold':'class'}, inplace=True)

for at in attributes[1:]:
    try:
        X[at] = pd.qcut(X[at],5,labels=False)
    except:
        continue

train,test=train_test_split(X,random_state=2017,test_size=0.4)

In [40]:
pd.value_counts(train['class'])

0    5352
1     969
Name: class, dtype: int64

In [41]:
pd.value_counts(test['class'])

0    3582
1     633
Name: class, dtype: int64

In [42]:
hc = HillClimbSearch(X, scoring_method=BicScore(X))
best_model = hc.estimate()

In [43]:
print "The edges are"
for edge in best_model.edges():
    print edge

The edges are
('Total Population', 'MHI')
('Total Population', 'n_tobacco')
('MHI', 'n_school')
('MHI', 'class')
('Percent White', 'Percent Hispanic')
('Percent White', 'Percent Asian')
('Percent White', '% Poverty')
('Percent White', 'Enrollment')
('Percent White', '% Female')
('Enrollment', 'Percent Asian')
('Enrollment', '% Poverty')
('Enrollment', 'subway_distance')
('Enrollment', 'Percent Black')
('Percent Asian', 'Percent Black')
('Percent Asian', 'school_distance')
('Percent Hispanic', 'Percent Asian')
('Percent Hispanic', 'Percent Black')
('% Female', 'Percent Hispanic')
('% Female', 'Enrollment')
('% Female', '% Poverty')
('% Female', 'subway_distance')
('n_tobacco', 'MHI')
('% Poverty', 'Percent Hispanic')
('% Poverty', 'subway_distance')
('subway_distance', 'school_distance')


In [59]:
X1 = X[['Total Population','MHI','n_tobacco','n_school','class']]
hc1 = HillClimbSearch(X1, scoring_method=BicScore(X1))
best_model1 = hc1.estimate()

In [61]:
model1 = BayesianModel(best_model1.edges())
print "The edges are"
for edge in best_model1.edges():
    print edge
model1.fit(X1, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)

The edges are
('Total Population', 'MHI')
('MHI', 'n_school')
('MHI', 'class')
('n_tobacco', 'Total Population')
('n_tobacco', 'MHI')


In [65]:
Xtest = test.loc[:,['Total Population','MHI','n_tobacco','n_school']]
Xtest = Xtest.dropna()
for col in Xtest.columns:
    Xtest[col] = Xtest[col].apply(int)

In [66]:
y_prob = model1.predict_probability(Xtest)

In [67]:
y_prob['class'] = y_prob.apply(lambda x: 0 if x['class_0'] > x['class_1'] else 1, axis=1)

In [68]:
accuracy = sum(y_prob['class']==test['class'])*1./len(y_prob)
fscore = f1_score(test['class'], y_prob['class'], average="macro")
precision = precision_score(test['class'], y_prob['class'], average="macro")
recall = recall_score(test['class'], y_prob['class'], average="macro")   

print "OS accuracy: " + str(accuracy)
print "OS precision: " + str(precision)
print "OS recall: " + str(recall)
print "OS f1-score: " + str(fscore)


OS accuracy: 0.849822064057
OS precision: 0.424911032028
OS recall: 0.5
OS f1-score: 0.459407464409


C:\Users\Nurvirta\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Nurvirta\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
y_prob.to_csv('y_prob_binary.csv')

# Anomaly

In [ ]:
# df['class'] = df['n_report'].apply(lambda x: 0 if x == 0 
#                                     else 1 if x == 1 
#                                     else 2 if x == 2 
#                                     else 3 if x >2 and x<=6 else 4)
df = pd.read_csv('New_Classes.csv', index_col=0)
attributes = ['class', 'MHI','Total Population', 'n_tobacco', 'school_distance', 'subway_distance', 'Enrollment', 
              '% Female'
             ]
# attributes = ['have_sold', 'MHI','Total Population', 'n_tobacco', 'school_distance', 'subway_distance', 'Enrollment', 
#               '% Female'
#              ]

X = df[attributes].copy()

# comment below
#X.rename(columns={'have_sold':'class'}, inplace=True)

for at in attributes[1:]:
    X[at] = pd.qcut(X[at],5,labels=False)

train,test=train_test_split(X,random_state=2017,test_size=0.4)

In [ ]:
pd.value_counts(train['class'])

In [ ]:
hc = HillClimbSearch(X, scoring_method=BicScore(X))
best_model = hc.estimate()

In [ ]:
print "The edges are"
for edge in best_model.edges():
    print edge

In [ ]:
model = BayesianModel(best_model.edges())

model.fit(X, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)

In [ ]:
Xtest = test.iloc[:,1:]
y_prob = model.predict_probability(Xtest)

In [ ]:
y_prob.to_csv('y_prob_multiclass.csv')